In [18]:
import os
import sys

import pandas as pd
import numpy as np

import scipy.stats as stats
import statsmodels.api as sm 
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [19]:
# Read in data
os.chdir("/Users/andreamendoza/Desktop/ECO 726")
os.getcwd()
os.path.dirname(os.getcwd())
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, 'data')
DATA_DIR
csv_file = os.path.join(DATA_DIR, 'bracero_aer_dataset_prep.csv')
df = pd.read_csv(csv_file)
df.shape

(15831, 105)

In [20]:
# Recreate table 2: Bracero on employment 
# print(res.summary(slim=True))

In [21]:
# Convert time_m from string to int for regression analysis
df['year'] = df['time_m'].str.slice(0, 4).astype(int)
df['month'] = df['time_m'].str.slice(5).astype(int)

df = df.sort_values(["State_FIPS", "year", "month"]).reset_index(drop=True)
df["time_num"] = df.groupby("State_FIPS").cumcount() + 1
print(df[["State_FIPS", "time_m", "year", "month", "time_num"]].head(600))

     State_FIPS   time_m  year  month  time_num
0             1   1942m1  1942      1         1
1             1   1942m4  1942      4         2
2             1   1942m7  1942      7         3
3             1  1942m10  1942     10         4
4             1   1943m1  1943      1         5
..          ...      ...   ...    ...       ...
595           4   1966m9  1966      9       271
596           4  1966m10  1966     10       272
597           4  1966m11  1966     11       273
598           4  1966m12  1966     12       274
599           4   1967m1  1967      1       275

[600 rows x 5 columns]


In [22]:
def get_significance_stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "**"
    elif p < 0.10:
        return "*"
    else:
        return ""

In [23]:
# store treatment_frac regression estimates
def extract_tf(res):
    # Convert numpy arrays to Series with variable names
    params = pd.Series(res.params, index=res.model.exog_names)
    tvals  = pd.Series(res.tvalues, index=res.model.exog_names)
    pvals  = pd.Series(res.pvalues, index=res.model.exog_names)

    coef = params["treatment_frac"]
    tval = tvals["treatment_frac"]
    pval = pvals["treatment_frac"]

    stars = get_significance_stars(pval)

    return {
        "coef": f"{coef:.3f}{stars}",
        "t": f"({tval:.2f})",
        "N": int(res.nobs),
    }

In [24]:
# All states, all years
# linear
df_clean = df.dropna(subset=['domestic_seasonal', 'treatment_frac', 'State_FIPS', 'time_num'])
formula = "domestic_seasonal ~ treatment_frac + C(State_FIPS) + C(time_num)"
res1 = smf.ols(formula, data=df_clean).fit()
res = res1.get_robustcov_results(cov_type='cluster', groups=df_clean['State_FIPS'])

#ln 
df_clean_ln = df.dropna(subset=['ln_domestic_seasonal', 'treatment_frac', 'State_FIPS', 'time_num'])
formula = "ln_domestic_seasonal ~ treatment_frac + C(State_FIPS) + C(time_num)"
res2 = smf.ols(formula, data=df_clean_ln).fit()
res = res2.get_robustcov_results(cov_type='cluster', groups=df_clean_ln['State_FIPS'])

In [25]:
# All states, years 1960-1970
#linear
df_clean_dom = df.dropna(subset=['domestic_seasonal', 'treatment_frac', 'State_FIPS', 'time_num'])
df_60_70 = df_clean_dom[(df_clean_dom['Year'] >= 1960) & (df_clean_dom['Year'] <= 1970)]

formula = "domestic_seasonal ~ treatment_frac + C(State_FIPS) + C(time_num)"
res3 = smf.ols(formula, data=df_60_70).fit()
res = res3.get_robustcov_results(cov_type='cluster', groups=df_60_70['State_FIPS'])

#ln
df_clean_ln = df.dropna(subset=['ln_domestic_seasonal', 'treatment_frac', 'State_FIPS', 'time_num'])
df_60_70_ln = df_clean_ln[(df_clean_ln['Year'] >= 1960) & (df_clean_ln['Year'] <= 1970)]

formula = "ln_domestic_seasonal ~ treatment_frac + C(State_FIPS) + C(time_num)"
res4 = smf.ols(formula, data=df_60_70_ln).fit()
res = res4.get_robustcov_results(cov_type='cluster', groups=df_60_70_ln['State_FIPS'])

In [26]:
# Exposed states only, all years
# linear
df_clean_dom = df.dropna(subset=['domestic_seasonal', 'treatment_frac', 'State_FIPS', 'time_num'])
df_not_none = df_clean_dom[df['none'] == 0].copy()

formula = "domestic_seasonal ~ treatment_frac + C(State_FIPS) + C(time_num)"
res5 = smf.ols(formula, data=df_not_none).fit()
res = res5.get_robustcov_results(cov_type='cluster', groups=df_not_none['State_FIPS'])

#ln
df_clean_dom_ln = df.dropna(subset=['ln_domestic_seasonal', 'treatment_frac', 'State_FIPS', 'time_num'])
df_not_none = df_clean_dom_ln[df['none'] == 0].copy()

formula = "ln_domestic_seasonal ~ treatment_frac + C(State_FIPS) + C(time_num)"
res6 = smf.ols(formula, data=df_not_none).fit()
res = res6.get_robustcov_results(cov_type='cluster', groups=df_not_none['State_FIPS'])

/var/folders/k1/5f3jlncn47xf0cm53z9ktclh0000gn/T/ipykernel_97180/3077676412.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_not_none = df_clean_dom[df['none'] == 0].copy()
/var/folders/k1/5f3jlncn47xf0cm53z9ktclh0000gn/T/ipykernel_97180/3077676412.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_not_none = df_clean_dom_ln[df['none'] == 0].copy()


In [27]:
out1 = extract_tf(res1)
out2 = extract_tf(res2)
out3 = extract_tf(res3)
out4 = extract_tf(res4)
out5 = extract_tf(res5)
out6 = extract_tf(res6)

from tabulate import tabulate
results = {
    "Linear, all years": extract_tf(res1),
    "ln, all years": extract_tf(res2),
    "linear, 1960-1970": extract_tf(res3),
    "ln, 1960-1970": extract_tf(res4),
    "linear, exposed states": extract_tf(res5),
    "ln, exposed states": extract_tf(res6),
}
print("Effects of Bracero Exclusion on Domestic seasonal ag employment, monthly\n")
df_table = pd.DataFrame(results)
print(tabulate(df_table, headers="keys", tablefmt="github"))

Effects of Bracero Exclusion on Domestic seasonal ag employment, monthly

|      | Linear, all years   | ln, all years   | linear, 1960-1970   | ln, 1960-1970   | linear, exposed states   | ln, exposed states   |
|------|---------------------|-----------------|---------------------|-----------------|--------------------------|----------------------|
| coef | -6949.211**         | -0.311*         | 1842.959            | -0.113          | 312.195                  | -0.142               |
| t    | (-2.41)             | (-1.73)         | (0.51)              | (-0.50)         | (0.09)                   | (-0.73)              |
| N    | 10329               | 6386            | 6072                | 3707            | 5168                     | 3189                 |


In [28]:
# Recreate table 3: Bracero on types of domestic employment 
# print(res.summary(slim=True))

In [29]:
#linear, local 
df_clean_local = df.dropna(subset=['Local_final', 'treatment_frac', 'State_FIPS', 'time_num'])
formula = "Local_final ~ treatment_frac + C(State_FIPS) + C(time_num)"
res7 = smf.ols(formula, data=df_clean_local).fit()
res = res7.get_robustcov_results(cov_type='cluster', groups=df_clean_local['State_FIPS'])

#linear, intrastate 
df_clean_intra = df.dropna(subset=['Intrastate_final', 'treatment_frac', 'State_FIPS', 'time_num'])
formula = "Intrastate_final ~ treatment_frac + C(State_FIPS) + C(time_num)"
res8 = smf.ols(formula, data=df_clean_intra).fit()
res = res8.get_robustcov_results(cov_type='cluster', groups=df_clean_intra['State_FIPS'])

#linear, interstate 
df_clean_inter = df.dropna(subset=['Interstate_final', 'treatment_frac', 'State_FIPS', 'time_num'])
formula = "Interstate_final ~ treatment_frac + C(State_FIPS) + C(time_num)"
res9 = smf.ols(formula, data=df_clean_inter).fit()
res = res9.get_robustcov_results(cov_type='cluster', groups=df_clean_inter['State_FIPS'])

In [30]:
#ln, local 
df_clean_local_ln = df.dropna(subset=['ln_local', 'treatment_frac', 'State_FIPS', 'time_num'])
formula = "ln_local ~ treatment_frac + C(State_FIPS) + C(time_num)"
res10 = smf.ols(formula, data=df_clean_local_ln).fit()
res = res10.get_robustcov_results(cov_type='cluster', groups=df_clean_local_ln['State_FIPS'])  

#ln, intrastate
df_clean_intra_ln = df.dropna(subset=['ln_intrastate', 'treatment_frac', 'State_FIPS', 'time_num'])
formula = "ln_intrastate ~ treatment_frac + C(State_FIPS) + C(time_num)"
res11 = smf.ols(formula, data=df_clean_intra_ln).fit()
res = res11.get_robustcov_results(cov_type='cluster', groups=df_clean_intra_ln['State_FIPS'])   

#ln, interstate
df_clean_inter_ln = df.dropna(subset=['ln_interstate', 'treatment_frac', 'State_FIPS', 'time_num'])
formula = "ln_interstate ~ treatment_frac + C(State_FIPS) + C(time_num)"
res12 = smf.ols(formula, data=df_clean_inter_ln).fit()
res = res12.get_robustcov_results(cov_type='cluster', groups=df_clean_inter_ln['State_FIPS'])

In [31]:
out7 = extract_tf(res7)
out8 = extract_tf(res8)
out9 = extract_tf(res9)
out10 = extract_tf(res10)
out11 = extract_tf(res11)
out12 = extract_tf(res12)

from tabulate import tabulate
results = {
    "linear, local": extract_tf(res7),
    "linear, intrastate": extract_tf(res8),
    "linear, interstate": extract_tf(res9),
    "ln, local": extract_tf(res10),
    "ln, intrastate": extract_tf(res11),
    "ln, interstate": extract_tf(res12),
}
print("Effects of Bracero Exclusion on Domestic seasonal ag employment, monthly\n")
df_table = pd.DataFrame(results)
print(tabulate(df_table, headers="keys", tablefmt="github"))

Effects of Bracero Exclusion on Domestic seasonal ag employment, monthly

|      | linear, local   | linear, intrastate   | linear, interstate   | ln, local   | ln, intrastate   | ln, interstate   |
|------|-----------------|----------------------|----------------------|-------------|------------------|------------------|
| coef | -2971.334       | -9083.240***         | 578.743              | -0.472***   | -0.997***        | -0.574**         |
| t    | (-1.28)         | (-11.20)             | (0.87)               | (-2.71)     | (-3.80)          | (-2.10)          |
| N    | 10329           | 6370                 | 6371                 | 6736        | 4720             | 5773             |
